装饰器本质上是一个 Python 函数或类，  
它可以让其他函数或类在不需要做任何代码修改的前提下增加额外功能，装饰器的返回值也是一个函数/类对象。  
它经常用于有切面需求的场景，比如：插入日志、性能测试、事务处理、缓存、权限校验等场景，装饰器是解决这类问题的绝佳设计。  
有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码到装饰器中并继续重用。  
概括的讲，装饰器的作用就是为已经存在的对象添加额外的功能。

## 简单装饰器
use_logging 就是一个装饰器，它一个普通的函数，它把执行真正业务逻辑的函数 func 包裹在其中，看起来像 foo 被 use_logging 装饰了一样，use_logging 返回的也是一个函数，这个函数的名字叫 wrapper。在这个例子中，函数进入和退出时 ，被称为一个横切面，这种编程方式被称为面向切面的编程。


In [11]:
import logging
def use_logging(func):
    def wrapper():
        logging.warning("%s is running" % func.__name__)
        return func()   # 把 foo 当做参数传递进来时，执行func()就相当于执行foo()
    return wrapper

def foo():
    print('i am foo')

foo = use_logging(foo)  # 因为装饰器 use_logging(foo) 返回的时函数对象 wrapper，这条语句相当于  foo = wrapper
foo()                   # 执行foo()就相当于执行 wrapper()

i am foo


## @ 语法糖
如果你接触 Python 有一段时间了的话，想必你对 @ 符号一定不陌生了，没错 @ 符号就是装饰器的语法糖，它放在函数开始定义的地方，这样就可以省略最后一步再次赋值的操作。

其实就是 foo = use_logging(foo) # 即 foo 作为参数传递给 use_logging。

此后的 foo 是装饰器中的 wrapper 函数对象了，而不是原来的函数的名称。当调用 foo 的时候，其实调用的是 wrapper() 函数,而在dec函数内部，又调用了  foo ，这样就造成了装饰器的效果。

这也解释了 foo 是被装饰函数，*arg是被装饰函数的参数—这种现象了。



In [14]:
def use_logging(func):
    def wrapper():
        logging.warning("%s is running" % func.__name__)
        return func()
    return wrapper

@use_logging
def foo():
    print("i am foo")

foo()

@use_logging
def duo():
    print('I am duo')

duo()

i am foo
I am duo


如上所示，有了 @ ，我们就可以省去foo = use_logging(foo)这一句了，直接调用 foo() 即可得到想要的结果。你们看到了没有，foo() 函数不需要做任何修改，只需在定义的地方加上装饰器，调用的时候还是和以前一样，如果我们有其他的类似函数，我们可以继续调用装饰器来修饰函数，而不用重复修改函数或者增加新的封装。这样，我们就提高了程序的可重复利用性，并增加了程序的可读性。

装饰器在 Python 使用如此方便都要归因于 Python 的函数能像普通的对象一样能作为参数传递给其他函数，可以被赋值给其他变量，可以作为返回值，可以被定义在另外一个函数内。

## *args、**kwargs
可能有人问，如果我的业务逻辑函数 foo 需要参数怎么办？  
当装饰器不知道 foo 到底有多少个参数时，我们可以用 *args 来代替
如果 foo 函数还定义了一些关键字参数,我们可以用 **kargs 来代替

In [29]:
def use_logging(func):
    def wrapper(*args,**kwargs):
        logging.warning("%s is running" % func.__name__)
        return func(*args,**kwargs)
    return wrapper

@use_logging
def foo(name):
    print("i am %s" % name)

foo('Michael')

i am Michael


## 带参数的装饰器
带参数的decorator，作用是通过传递参数可以定制不同的装饰器。  
装饰器还有更大的灵活性，例如带参数的装饰器，在上面的装饰器调用中，该装饰器接收唯一的参数就是执行业务的函数 foo 。  
装饰器的语法允许我们在调用时，提供其它参数，  
比如@decorator(a)  
这样，就为装饰器的编写和使用提供了更大的灵活性。  
比如，我们可以在装饰器中指定日志的等级，因为不同业务函数可能需要的日志级别是不一样的。


In [21]:
def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warning("%s is running" % func.__name__)
            elif level == "info":
                logging.info("%s is running" % func.__name__)
            return func(*args)
        return wrapper

    return decorator

@use_logging(level="info")
def foo(name='foo'):
    print("i am %s" % name)

foo()

i am foo


内部逻辑为： foo = use_logging('info')(foo)

这里 use_logging('info') 返回是 decorator 函数对象，  
这样的话，use_logging('info')(foo) 其实就是 decorator(foo)，这样就和上面的一样了。
只不过这里传递了一个参数 'info' 进入decorator内部，使得我们可以操作这个参数。


上面的 use_logging 是允许带参数的装饰器。它实际上是对原有装饰器的一个函数封装，并返回一个装饰器。我们可以将它理解为一个含有参数的闭包。当我 们使用@use_logging(level="warn")调用的时候，Python 能够发现这一层的封装，并把参数传递到装饰器的环境中。

@use_logging(level="warn")等价于@decorator

## Function Decorator
Let us now, write a simple function decorator for ourselves. We will write a decorator that would measure the execution time of the function passed to it.

In [4]:
import time

def timetest(input_func):

    def timed(*args, **kwargs):
        start_time = time.time()
        result = input_func(*args, **kwargs)
        end_time = time.time()
        print("Method Name - {0}, Args - {1}, Kwargs - {2}, Execution Time - {3}".format(
            input_func.__name__,
            args,
            kwargs,
            end_time - start_time))
        return result
    return timed

@timetest
def foobar(*args, **kwargs):
    time.sleep(0.3)
    print("inside foobar")
    print(args, kwargs)

In [5]:
foobar(["hello, world"], foo=2, bar=5)

inside foobar
(['hello, world'],) {'foo': 2, 'bar': 5}
Method Name - foobar, Args - (['hello, world'],), Kwargs - {'foo': 2, 'bar': 5}, Execution Time - 0.3066873550415039


We passed the function foobar to decorator named timetest. Inside decorator, function foobar is referenced as variable input_func. The result, post execution of input_func is referred as result.

Prepending @ to the name of the decorator, and writing the same above a function calls the decorator, and passes the function to the decorator(decorates).

## Method Decorator
函数decorator可以修饰类成员函数  
Method decorators allow overriding class properties by decorating, without having to find the calling function.

in the snippet shown above, we decorate the class method print_test. The method_decorator prints the name of the city, if the name of city instance is not SFO.

原理：注意此时 print_test 的self会被传递到 method_decorator 中。此时把self看做普通的函数入参。

In [8]:
def method_decorator(method):
    def inner(city_instance):
        if city_instance.name == "SFO":
            print ("Its a cool place to live in.")
        else:
            method(city_instance)
    return inner


class City(object):
    def __init__(self, name):
        self.name = name

    @method_decorator
    def print_test(self):
        print (self.name)

p1 = City("SFA")

p1.print_test()

SFA


## @property
Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [30]:
class Student(object):

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，  
此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，  
于是，我们就拥有一个可控的属性操作：

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

In [35]:
s =Student()
s.score=60
s.score

60

## 类装饰器
没错，装饰器不仅可以是函数，还可以是类，相比函数装饰器，类装饰器具有灵活度大、高内聚、封装性等优点。  
使用类装饰器主要依靠类的__call__方法，当使用 @ 形式将装饰器附加到函数上时，就会调用此方法。


In [22]:
class Foo(object):
    def __init__(self, func):
        self._func = func

    def __call__(self):
        print ('class decorator runing')
        self._func()
        print ('class decorator ending')

@Foo
def bar():
    print ('bar')

bar()

class decorator runing
bar
class decorator ending


## functools.wraps
使用装饰器极大地复用了代码，但是他有一个缺点就是原函数的元信息不见了，比如函数的docstring、__name__、参数列表，先看例子：


In [28]:
# 装饰器
def logged(func):
    def with_logging(*args, **kwargs):
        print (func.__name__)     # 输出 'with_logging'
        print (func.__doc__ )      # 输出 None
        return func(*args, **kwargs)
    return with_logging

# 函数
@logged
def f(x):
   """does some math"""
   return x + x * x

f(5)

f
does some math


30

## 装饰器顺序
一个函数还可以同时定义多个装饰器，比如：  
它的执行顺序是从里到外，最先调用最里层的装饰器，最后调用最外层的装饰器，它等效于

f = a(b(c(f)))

In [ ]:
@a
@b
@c
def f ():
    pass
